# LSTM-only model 

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import math
import shutil
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Masking
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot
import sys
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load a single file as a numpy array
import pandas as pd
import numpy as np
import re

def load_file(filepath):
  tmpList= []
  tmpList1 = []
  tmpList2 = []
  with open(filepath,'r') as thecsv:
    for line in thecsv:
      line = re.sub(re.compile(r'\s+'), '', line)
      line = line.strip(",")
      tmpList.append(line.split(',')[0:24])

    for i in range(len(tmpList)):
      tmpList1 = []
      for element in tmpList[i]:
        tmpList1.append((float(element)))
      tmpList2.append(tmpList1)
  dataframe2 = pd.DataFrame(tmpList2)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3.csv')
  dataframe2 = dataframe2.replace(np.nan,0.0, regex=True)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3with0.csv')
  dataframe = dataframe2.astype(float)
  return dataframe.values

def load_file_y(filepath):
  dataframe_y = read_csv(filepath, header=None)
  file = open(filepath, "r")
  csv_reader = csv. reader(file)
  lists_from_csv = []
  for row in csv_reader:
    lists_from_csv. append(int(row[0])-1)
  # return dataframe_y.values.tolist()
  return lists_from_csv

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data) 
  # stack group so that features are the 3rd dimension
  # print("+++++++++++++++++++++++++\n\nBEFORE DSTACK\n\n++++++++++++++++++")
  loaded = dstack(loaded)
  # print("+++++++++++++++++++++++++\n\nAFTER DSTACK\n\n++++++++++++++++++")
  # print("(Total samples, Timesteps, features) - ", loaded.shape)
  # sys.exit(0)
  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + "/"
  filenames = list()
  filenames += ['leftshoulder_velocity.txt','leftshoulder_acceleration.txt','leftelbow_velocity.txt','leftelbow_acceleration.txt','leftwrist_velocity.txt', 'leftwrist_acceleration.txt',]
  filenames += ['rightshoulder_velocity.txt','rightshoulder_acceleration.txt','rightelbow_velocity.txt','rightelbow_acceleration.txt','rightwrist_velocity.txt', 'rightwrist_acceleration.txt',]
  # filenames += ['leftshoulder_polarangle.txt','leftshoulder_velocity.txt','leftelbow_polarangle.txt','leftelbow_velocity.txt','leftwrist_polarangle.txt', 'leftwrist_velocity.txt']
  # filenames += ['rightshoulder_polarangle.txt','rightshoulder_velocity.txt','rightelbow_polarangle.txt','rightelbow_velocity.txt','rightwrist_polarangle.txt', 'rightwrist_velocity.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file_y( './drive/My Drive/CS298/2D' + '/y_9gestures.txt')
  return X, y

In [ ]:
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit



# load the dataset, returns train and test X and y elements
def load_dataset(prefix='./drive/My Drive/CS298/2D/angular_acceleration_6joints_9gestures'):
  # load all train
  # trainX, trainy = load_dataset_group('train', prefix)
  # load all test
  X, y = load_dataset_group('doesntmatter', prefix)
  y = np.array(y)
  sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=42)
  for train_index, test_index in sss.split(X, y):
    # print("TRAIN:", train_index, "TEST:", test_index)
    trainX, testX = X[train_index], X[test_index]
    trainy, testy = y[train_index], y[test_index]
  # trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=42)
  # zero-offset class values
  # trainy = trainy - 1
  # testy = testy - 1
  # one hot encode y
  #print("Before_Categorization",trainy.shape,testy.shape)
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  #print("After_Categorization",trainy.shape,testy.shape)
  return trainX, trainy, testX, testy

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  #better result for 50,271
  verbose, epochs, batch_size = 0, 50, 271
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  print("Timesteps - ",n_timesteps, "Features - ",n_features, "Output Dimension - ",n_outputs)
  model = tf.keras.models.Sequential()
  model.add(Dropout(0.5))
  model.add(tf.keras.layers.Masking(mask_value=0.0,input_shape=(n_timesteps, n_features)))
  model.add(tf.keras.layers.LSTM(543))#######need to add the code to directly pick sample -> now it is manually written as 117
  model.add(Dense(1000, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # Confusion matrix
  predictions = model.predict(testX)
  # matrix = confusion_matrix(testy.argmax(axis=1), predictions.argmax(axis=1))
  # print(matrix)
  print(classification_report(testy.argmax(axis=1), predictions.argmax(axis=1)))
  # evaluate model
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=2)
  # _, accuracy = model.evaluate(testX, testy, verbose=0)
  # accuracy = 0
  return accuracy

In [ ]:
# summarize scores
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores)
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=3):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  # print(trainX.shape, trainy.shape, testX.shape,testy.shape)
  scores = list()
  for r in range(repeats):
    print("\n")
    print("Running LSTM with Angular velocity and Acceleration - 9 gestures")
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [ ]:
run_experiment()



Running LSTM with Angular velocity and Acceleration - 9 gestures
Timesteps -  24 Features -  12 Output Dimension -  9
              precision    recall  f1-score   support

           0       0.39      0.23      0.29        30
           1       0.48      0.43      0.46        30
           2       0.13      0.06      0.09        31
           3       0.30      0.41      0.35        32
           4       0.43      0.84      0.57        31
           5       0.12      0.17      0.14        30
           6       0.58      0.25      0.35        28
           7       0.23      0.37      0.28        30
           8       0.25      0.07      0.11        30

    accuracy                           0.32       272
   macro avg       0.32      0.31      0.29       272
weighted avg       0.32      0.32      0.29       272

2/2 - 2s - loss: 1.8060 - accuracy: 0.3162 - 2s/epoch - 864ms/step
>#1: 31.618


Running LSTM with Angular velocity and Acceleration - 9 gestures
Timesteps -  24 Features -  1

# CNN LSTM

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from matplotlib import pyplot
from tensorflow.keras.utils import to_categorical

In [ ]:
# load a single file as a numpy array
import pandas as pd
import numpy as np
import re

def load_file(filepath):
  tmpList= []
  tmpList1 = []
  tmpList2 = []
  with open(filepath,'r') as thecsv:
    for line in thecsv:
      line = re.sub(re.compile(r'\s+'), '', line)
      line = line.strip(",")
      tmpList.append(line.split(',')[0:24])

    for i in range(len(tmpList)):
      tmpList1 = []
      for element in tmpList[i]:
        tmpList1.append((float(element)))
      tmpList2.append(tmpList1)
  dataframe2 = pd.DataFrame(tmpList2)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3.csv')
  dataframe2 = dataframe2.replace(np.nan,0.0, regex=True)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3with0.csv')
  dataframe = dataframe2.astype(float)
  return dataframe.values

def load_file_y(filepath):
  dataframe_y = read_csv(filepath, header=None)
  file = open(filepath, "r")
  csv_reader = csv. reader(file)
  lists_from_csv = []
  for row in csv_reader:
    lists_from_csv. append(int(row[0])-1)
  # return dataframe_y.values.tolist()
  return lists_from_csv

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)

  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  print("(Total samples, Timesteps, features) - ", loaded.shape)

  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + "/"
  filenames = list()
  filenames += ['leftshoulder_velocity.txt','leftshoulder_acceleration.txt','leftelbow_velocity.txt','leftelbow_acceleration.txt','leftwrist_velocity.txt', 'leftwrist_acceleration.txt',]
  filenames += ['rightshoulder_velocity.txt','rightshoulder_acceleration.txt','rightelbow_velocity.txt','rightelbow_acceleration.txt','rightwrist_velocity.txt', 'rightwrist_acceleration.txt',]
  # filenames += ['leftshoulder_polarangle.txt','leftshoulder_velocity.txt','leftelbow_polarangle.txt','leftelbow_velocity.txt','leftwrist_polarangle.txt', 'leftwrist_velocity.txt']
  # filenames += ['rightshoulder_polarangle.txt','rightshoulder_velocity.txt','rightelbow_polarangle.txt','rightelbow_velocity.txt','rightwrist_polarangle.txt', 'rightwrist_velocity.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file_y( './drive/My Drive/CS298/2D' + '/y_9gestures.txt')
  # print("Printing X -----------------------")
  # print(X)
  # print("Printing y -----------------------")
  return X, y

In [ ]:
from sklearn.model_selection import train_test_split



# load the dataset, returns train and test X and y elements
def load_dataset(prefix='./drive/My Drive/CS298/2D/angular_acceleration_6joints_9gestures'):
  # load all train
  # trainX, trainy = load_dataset_group('train', prefix)
  # load all test
  X, y = load_dataset_group('doesntmatter', prefix)
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.25, random_state=42)

  # zero-offset class values
  #trainy = trainy - 1
  #testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)


  return trainX, trainy, testX, testy

In [ ]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  verbose, epochs, batch_size = 0, 100, 60
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  print("Timesteps - ",n_timesteps, "Features - ",n_features, "Output Dimension - ",n_outputs)
  n_steps, n_length = 2,12
  trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
  testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
	# define model
  model = Sequential()
  model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
  # model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
  model.add(TimeDistributed(Dropout(0.5)))
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(500))
  model.add(Dropout(0.5))
  model.add(Dense(100, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
  predictions = model.predict(testX)
  # matrix = confusion_matrix(testy.argmax(axis=1), predictions.argmax(axis=1))
  # print(matrix)
  print(classification_report(testy.argmax(axis=1), predictions.argmax(axis=1)))
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy
 

In [ ]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=3):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  scores = list()
  for r in range(repeats):
    print("\n")
    print("Running CNN-LSTM with Angular velocity and Acceleration-2D - 9 gestures")
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [ ]:
run_experiment()

(Total samples, Timesteps, features) -  (543, 24, 12)


Running CNN-LSTM with Angular velocity and Acceleration-2D - 9 gestures
Timesteps -  24 Features -  12 Output Dimension -  9
              precision    recall  f1-score   support

           0       0.80      0.29      0.42        14
           1       0.75      0.75      0.75        20
           2       0.40      0.46      0.43        13
           3       0.60      0.80      0.69        15
           4       0.92      0.92      0.92        13
           5       0.36      0.31      0.33        13
           6       0.92      0.80      0.86        15
           7       0.48      0.93      0.63        14
           8       0.67      0.42      0.52        19

    accuracy                           0.63       136
   macro avg       0.66      0.63      0.62       136
weighted avg       0.66      0.63      0.62       136

>#1: 63.235


Running CNN-LSTM with Angular velocity and Acceleration-2D - 9 gestures
Timesteps -  24 Features -  

# ConvLSTM

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [ ]:
# load a single file as a numpy array
import pandas as pd
import numpy as np
import re

def load_file(filepath):
  tmpList= []
  tmpList1 = []
  tmpList2 = []
  with open(filepath,'r') as thecsv:
    for line in thecsv:
      line = re.sub(re.compile(r'\s+'), '', line)
      line = line.strip(",")
      tmpList.append(line.split(',')[0:24])

    for i in range(len(tmpList)):
      tmpList1 = []
      for element in tmpList[i]:
        tmpList1.append((float(element)))
      tmpList2.append(tmpList1)
  dataframe2 = pd.DataFrame(tmpList2)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3.csv')
  dataframe2 = dataframe2.replace(np.nan,0.0, regex=True)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3with0.csv')
  dataframe = dataframe2.astype(float)
  return dataframe.values

def load_file_y(filepath):
  dataframe_y = read_csv(filepath, header=None)
  file = open(filepath, "r")
  csv_reader = csv. reader(file)
  lists_from_csv = []
  for row in csv_reader:
    lists_from_csv. append(int(row[0])-1)
  # return dataframe_y.values.tolist()
  return lists_from_csv

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)

  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  print("(Total samples, Timesteps, features) - ", loaded.shape)

  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + "/"
  filenames = list()
  filenames += ['leftshoulder_velocity.txt','leftshoulder_acceleration.txt','leftelbow_velocity.txt','leftelbow_acceleration.txt','leftwrist_velocity.txt', 'leftwrist_acceleration.txt',]
  filenames += ['rightshoulder_velocity.txt','rightshoulder_acceleration.txt','rightelbow_velocity.txt','rightelbow_acceleration.txt','rightwrist_velocity.txt', 'rightwrist_acceleration.txt',]
  # filenames += ['leftshoulder_polarangle.txt','leftshoulder_velocity.txt','leftelbow_polarangle.txt','leftelbow_velocity.txt','leftwrist_polarangle.txt', 'leftwrist_velocity.txt']
  # filenames += ['rightshoulder_polarangle.txt','rightshoulder_velocity.txt','rightelbow_polarangle.txt','rightelbow_velocity.txt','rightwrist_polarangle.txt', 'rightwrist_velocity.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file_y( './drive/My Drive/CS298/2D' + '/y_9gestures.txt')
  # print("Printing X -----------------------")
  # print(X)
  # print("Printing y -----------------------")
  return X, y

In [ ]:
from sklearn.model_selection import train_test_split



# load the dataset, returns train and test X and y elements
def load_dataset(prefix='./drive/My Drive/CS298/2D/angular_acceleration_6joints_9gestures'):
  # load all train
  # trainX, trainy = load_dataset_group('train', prefix)
  # load all test
  X, y = load_dataset_group('doesntmatter', prefix)
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.25, random_state=42)

  # zero-offset class values
  #trainy = trainy - 1
  #testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)


  return trainX, trainy, testX, testy

In [ ]:
from keras.layers import LeakyReLU
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  verbose, epochs, batch_size = 0, 100, 60
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  # reshape into subsequences (samples, time steps, rows, cols, channels)
  n_steps, n_length = 2,12
  trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
  testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
  # define model
  model = Sequential()
  model.add(ConvLSTM2D(filters=30, kernel_size=(1,1), input_shape=(n_steps, 1, n_length, n_features)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(100,activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  predictions = model.predict(testX)
  print(classification_report(testy.argmax(axis=1),predictions.argmax(axis=1)))
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy

In [ ]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=3):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  scores = list()
  for r in range(repeats):
    print("\n")
    print("Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures")
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [ ]:
run_experiment()

(Total samples, Timesteps, features) -  (543, 24, 12)


Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures
              precision    recall  f1-score   support

           0       0.43      0.43      0.43        14
           1       0.72      0.65      0.68        20
           2       0.56      0.69      0.62        13
           3       0.75      0.60      0.67        15
           4       0.87      1.00      0.93        13
           5       0.38      0.46      0.41        13
           6       0.92      0.80      0.86        15
           7       0.52      0.79      0.63        14
           8       0.91      0.53      0.67        19

    accuracy                           0.65       136
   macro avg       0.67      0.66      0.65       136
weighted avg       0.69      0.65      0.66       136

>#1: 65.441


Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures
              precision    recall  f1-score   support

           0       0.57   

# ConvLSTM - LeakyReLU

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [ ]:
# load a single file as a numpy array
import pandas as pd
import numpy as np
import re

def load_file(filepath):
  tmpList= []
  tmpList1 = []
  tmpList2 = []
  with open(filepath,'r') as thecsv:
    for line in thecsv:
      line = re.sub(re.compile(r'\s+'), '', line)
      line = line.strip(",")
      tmpList.append(line.split(',')[0:24])

    for i in range(len(tmpList)):
      tmpList1 = []
      for element in tmpList[i]:
        tmpList1.append((float(element)))
      tmpList2.append(tmpList1)
  dataframe2 = pd.DataFrame(tmpList2)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3.csv')
  dataframe2 = dataframe2.replace(np.nan,0.0, regex=True)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3with0.csv')
  dataframe = dataframe2.astype(float)
  return dataframe.values

def load_file_y(filepath):
  dataframe_y = read_csv(filepath, header=None)
  file = open(filepath, "r")
  csv_reader = csv. reader(file)
  lists_from_csv = []
  for row in csv_reader:
    lists_from_csv. append(int(row[0])-1)
  # return dataframe_y.values.tolist()
  return lists_from_csv

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)

  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  print("(Total samples, Timesteps, features) - ", loaded.shape)

  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + "/"
  filenames = list()
  filenames += ['leftshoulder_velocity.txt','leftshoulder_acceleration.txt','leftelbow_velocity.txt','leftelbow_acceleration.txt','leftwrist_velocity.txt', 'leftwrist_acceleration.txt',]
  filenames += ['rightshoulder_velocity.txt','rightshoulder_acceleration.txt','rightelbow_velocity.txt','rightelbow_acceleration.txt','rightwrist_velocity.txt', 'rightwrist_acceleration.txt',]
  # filenames += ['leftshoulder_polarangle.txt','leftshoulder_velocity.txt','leftelbow_polarangle.txt','leftelbow_velocity.txt','leftwrist_polarangle.txt', 'leftwrist_velocity.txt']
  # filenames += ['rightshoulder_polarangle.txt','rightshoulder_velocity.txt','rightelbow_polarangle.txt','rightelbow_velocity.txt','rightwrist_polarangle.txt', 'rightwrist_velocity.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file_y( './drive/My Drive/CS298/2D' + '/y_9gestures.txt')
  # print("Printing X -----------------------")
  # print(X)
  # print("Printing y -----------------------")
  return X, y

In [ ]:
from sklearn.model_selection import train_test_split



# load the dataset, returns train and test X and y elements
def load_dataset(prefix='./drive/My Drive/CS298/2D/angular_acceleration_6joints_9gestures'):
  # load all train
  # trainX, trainy = load_dataset_group('train', prefix)
  # load all test
  X, y = load_dataset_group('doesntmatter', prefix)
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.25, random_state=42)

  # zero-offset class values
  #trainy = trainy - 1
  #testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)


  return trainX, trainy, testX, testy

In [ ]:
from keras.layers import LeakyReLU
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  verbose, epochs, batch_size = 0, 100, 60
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  # reshape into subsequences (samples, time steps, rows, cols, channels)
  n_steps, n_length = 2,12
  trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
  testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
  # define model
  model = Sequential()
  model.add(ConvLSTM2D(filters=30, kernel_size=(1,1), input_shape=(n_steps, 1, n_length, n_features)))
  model.add(Dropout(0.5))
  model.add(LeakyReLU(alpha=0.05))
  model.add(Flatten())
  model.add(Dense(100))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  predictions = model.predict(testX)
  print(classification_report(testy.argmax(axis=1),predictions.argmax(axis=1)))
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy

In [ ]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=3):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  scores = list()
  for r in range(repeats):
    print("\n")
    print("Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures")
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [ ]:
run_experiment()


(Total samples, Timesteps, features) -  (543, 24, 12)


Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures
              precision    recall  f1-score   support

           0       0.50      0.43      0.46        14
           1       0.76      0.65      0.70        20
           2       0.40      0.46      0.43        13
           3       0.71      0.67      0.69        15
           4       0.92      0.92      0.92        13
           5       0.53      0.69      0.60        13
           6       0.86      0.80      0.83        15
           7       0.73      0.79      0.76        14
           8       0.63      0.63      0.63        19

    accuracy                           0.67       136
   macro avg       0.67      0.67      0.67       136
weighted avg       0.68      0.67      0.67       136

>#1: 66.912


Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures
              precision    recall  f1-score   support

           0       0.53   

# ConvLSTM - LeakyReLU with increase in noise data

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot

In [ ]:
# load a single file as a numpy array
import pandas as pd
import numpy as np
import re

def load_file(filepath):
  tmpList= []
  tmpList1 = []
  tmpList2 = []
  with open(filepath,'r') as thecsv:
    for line in thecsv:
      line = re.sub(re.compile(r'\s+'), '', line)
      line = line.strip(",")
      tmpList.append(line.split(',')[0:44])

    for i in range(len(tmpList)):
      tmpList1 = []
      for element in tmpList[i]:
        tmpList1.append((float(element)))
      tmpList2.append(tmpList1)
  dataframe2 = pd.DataFrame(tmpList2)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3.csv')
  dataframe2 = dataframe2.replace(np.nan,0.0, regex=True)
  # dataframe2.to_csv('./drive/My Drive/Ashwini/gestureData/test1/dataframe3with0.csv')
  dataframe = dataframe2.astype(float)
  return dataframe.values

def load_file_y(filepath):
  dataframe_y = read_csv(filepath, header=None)
  file = open(filepath, "r")
  csv_reader = csv. reader(file)
  lists_from_csv = []
  for row in csv_reader:
    lists_from_csv. append(int(row[0])-1)
  # return dataframe_y.values.tolist()
  return lists_from_csv

In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)

  # stack group so that features are the 3rd dimension
  loaded = dstack(loaded)
  print("(Total samples, Timesteps, features) - ", loaded.shape)

  return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
  filepath = prefix + "/"
  filenames = list()
  filenames += ['leftshoulder_velocity.txt','leftshoulder_acceleration.txt','leftelbow_velocity.txt','leftelbow_acceleration.txt','leftwrist_velocity.txt', 'leftwrist_acceleration.txt',]
  filenames += ['rightshoulder_velocity.txt','rightshoulder_acceleration.txt','rightelbow_velocity.txt','rightelbow_acceleration.txt','rightwrist_velocity.txt', 'rightwrist_acceleration.txt',]
  # filenames += ['leftshoulder_polarangle.txt','leftshoulder_velocity.txt','leftelbow_polarangle.txt','leftelbow_velocity.txt','leftwrist_polarangle.txt', 'leftwrist_velocity.txt']
  # filenames += ['rightshoulder_polarangle.txt','rightshoulder_velocity.txt','rightelbow_polarangle.txt','rightelbow_velocity.txt','rightwrist_polarangle.txt', 'rightwrist_velocity.txt']
  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file_y( './drive/My Drive/CS298/2D' + '/y_9gestures.txt')
  # print("Printing X -----------------------")
  # print(X)
  # print("Printing y -----------------------")
  return X, y

In [ ]:
from sklearn.model_selection import train_test_split



# load the dataset, returns train and test X and y elements
def load_dataset(prefix='./drive/My Drive/CS298/2D/angular_acceleration_6joints_9gestures'):
  # load all train
  # trainX, trainy = load_dataset_group('train', prefix)
  # load all test
  X, y = load_dataset_group('doesntmatter', prefix)
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.25, random_state=42)

  # zero-offset class values
  #trainy = trainy - 1
  #testy = testy - 1
  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)


  return trainX, trainy, testX, testy

In [ ]:
from keras.layers import LeakyReLU
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  verbose, epochs, batch_size = 0, 100, 60
  n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
  # reshape into subsequences (samples, time steps, rows, cols, channels)
  n_steps, n_length = 2,22
  trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
  testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
  # define model
  model = Sequential()
  model.add(ConvLSTM2D(filters=30, kernel_size=(1,1), input_shape=(n_steps, 1, n_length, n_features)))
  model.add(Dropout(0.5))
  model.add(LeakyReLU(alpha=0.05))
  model.add(Flatten())
  model.add(Dense(100))
  model.add(Dense(n_outputs, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
  # evaluate model
  predictions = model.predict(testX)
  print(classification_report(testy.argmax(axis=1),predictions.argmax(axis=1)))
  _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
  return accuracy

In [ ]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
# run an experiment
def run_experiment(repeats=3):
  # load data
  trainX, trainy, testX, testy = load_dataset()
  scores = list()
  for r in range(repeats):
    print("\n")
    print("Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures")
    score = evaluate_model(trainX, trainy, testX, testy)
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  # summarize results
  summarize_results(scores)

In [ ]:
run_experiment()

(Total samples, Timesteps, features) -  (543, 44, 12)


Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures
              precision    recall  f1-score   support

           0       0.67      0.29      0.40        14
           1       0.74      0.70      0.72        20
           2       0.47      0.69      0.56        13
           3       0.62      0.67      0.65        15
           4       0.86      0.92      0.89        13
           5       0.40      0.62      0.48        13
           6       0.85      0.73      0.79        15
           7       0.76      0.93      0.84        14
           8       0.83      0.53      0.65        19

    accuracy                           0.67       136
   macro avg       0.69      0.67      0.66       136
weighted avg       0.70      0.67      0.67       136

>#1: 66.912


Running ConvLSTM with Angular velocity and Acceleration-2D - 9 gestures
              precision    recall  f1-score   support

           0       0.53   